In [1]:
# Imports
import pandas as pd
import gensim
import numpy as np
import nltk
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords
import pyLDAvis
import pyLDAvis.gensim

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from helpers import *
from LDA_helpers import *

np.random.seed(2018) # set random seed


In [2]:
# Load data
tweets1 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_1.csv')
tweets2 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_2.csv')
tweets3 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_3.csv')
# tweets4 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_4.csv') # process function throws error
tweets5 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_5.csv')
tweets6 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_6.csv')


tweets = pd.concat([tweets1, tweets2, tweets3, tweets5, tweets6], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)



In [3]:
tweets.shape

(1803466, 15)

In [4]:
data_hashtagged, tokens, hashtag_column = process_tweets(tweets, group_by='hashtag')

In [5]:
len(tokens)

90823

In [6]:
hashtag_column

0                                                          
1                                                          
2                                               #BoycottNFL
3                                                          
4                                        #StandForOurAnthem
5                                                          
6                                                          
7                                                          
8                                                          
9                                                          
10                                                         
11                                                         
12                                                         
13                                                         
14                                                         
15                                                         
16                                      

In [7]:
# type(hashtag_column)
hashtag_column.value_counts()

                                                                                                    738429
#news                                                                                                23191
#politics                                                                                            22569
#sports                                                                                              16661
#health                                                                                               8034
#business                                                                                             6764
#SanJose                                                                                              3971
#top                                                                                                  3780
#crime                                                                                                3502
#breaking                            

In [ ]:
ind_sport = data_hashtagged.index.get_loc("#sports")
ind_MAGA = data_hashtagged.index.get_loc('#MAGA')
ind_BlackLivesMatter = data_hashtagged.index.get_loc('#BlackLivesMatter')
ind_business = data_hashtagged.index.get_loc('#business')
ind_USFA = data_hashtagged.index.get_loc('#USFA')

# Concatenate the sublists into one to feed to the model
list_concat = []
list_concat.append(tokens[ind_sport])
list_concat.append(tokens[ind_MAGA])
list_concat.append(tokens[ind_BlackLivesMatter])
list_concat.append(tokens[ind_business])
list_concat.append(tokens[ind_USFA])
data_processed = list_concat.copy()

In [ ]:
# Create dictionary of sum of frequencies across all author IDs
dict1 = dict(hashtag_column.sum())
# Export into pandas for ordering
hashtag_frequencies = pd.DataFrame.from_dict(dict1, orient='index', columns=['count'])
hashtag_frequencies.sort_values(['count'], ascending=False)

In [ ]:
#sports, #MAGA, #BlackLivesMatter, #business, #USFA
# Load data
datafolder = './data/'
tweets1 = pd.read_csv(datafolder + 'IRAhandle_tweets_1.csv')
tweets2 = pd.read_csv(datafolder + 'IRAhandle_tweets_2.csv')
tweets = pd.concat([tweets1, tweets2], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)


In [ ]:
# Create dictionary to know the frequency of words
id2word = gensim.corpora.Dictionary(data_processed)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_processed]


In [ ]:
# Build LDA model
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=20, 
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)

lda_model = gensim.models.LdaMulticore(corpus = corpus,
                                       num_topics=6,
                                       id2word=id2word,
                                       chunksize=2000,
                                       passes=10, 
                                       per_word_topics=False,
                                       workers=2)

In [ ]:
print(lda_model.print_topics())

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_processed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
mallet_path = '/Users/halimaschede/Documents/GitHub/Adarmonic_mean_project/data/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=15, id2word=id2word)

# Show Topics
print(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=content_stemmed, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


In [ ]:
# Check shit is working
test = bow_corpus[126]
for i in range(len(test)):
    print("Word {} (\"{}\") appears {} time.".format(test[i][0],dictionary[test[i][0]], 
test[i][1]))

In [ ]:
# Create model on bag-of-words

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
print(lda_model.show_topic(0))
# for topic in range(10):
#     plt.figure()
#     plt.imshow(WordCloud().fit_words(lda_model.show_topic(topic)))
#     plt.axis("off")
#     plt.title("Topic #" + str(topic))
#     plt.show()
